In [1]:
import pandas as pd
import os

In [2]:
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [3]:
teams_df = pd.DataFrame([])
current_df = pd.DataFrame([])
for team_file in os.listdir('data/TeamsPrep'):
    team_id = team_file[:3]
    
    for year in [2020, 2021, 2022, 2023]:
        team_df = pd.read_excel(f'data/TeamsPrep/{team_file}', str(year))

        team_df['TeamID'] = mapTeamID(team_id)
        team_df['OppID'] = team_df['OppID'].apply(lambda x: mapTeamID(x))
        team_df['Season'] = year

        teams_df = pd.concat([teams_df, team_df])
        
    team_df = pd.read_excel(f'data/CurrentSeason/{team_file}')
    
    roll_df = getTeamRollingSeason(team_df)
    prep_df = dataPrep(roll_df, 'Games')
    
    prep_df['TeamID'] = mapTeamID(team_id)
    prep_df['OppID'] = prep_df['OppID'].apply(lambda x: mapTeamID(x))
    prep_df['Season'] = 2024
    
    current_df = pd.concat([current_df, prep_df])

In [4]:
teams_df = teams_df.sort_values(by=['Date']).reset_index(drop=True)
teams_df.head()

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppORB,OppFTR,OppORT,Location,Target,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,2,2019-10-24,1,119.0,100.2,0.596,16.3,27.8,0.333,118.8,...,21.1,0.209,120.9,1,0,10,3,1,9,2020
1,2,2019-10-24,1,112.0,95.3,0.586,13.3,25.6,0.210,117.5,...,29.4,0.293,115.7,0,1,10,3,2,13,2020
2,2,2019-10-25,-1,102.0,95.3,0.512,12.9,20.9,0.176,107.0,...,21.2,0.156,101.2,1,1,10,4,3,14,2020
3,2,2019-10-25,-1,95.0,108.7,0.448,20.9,25.5,0.195,87.4,...,27.7,0.241,100.1,1,0,10,4,2,26,2020
4,2,2019-10-25,-1,93.0,101.8,0.406,8.7,16.7,0.222,91.4,...,30.2,0.311,115.2,1,1,10,4,2,2,2020


In [5]:
current_df = current_df.sort_values(by=['Date']).reset_index(drop=True)
current_df.head()

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppORB,OppFTR,OppORT,Location,Target,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,2,2023-10-26,-1.0,107.0,95.3,0.511,10.0,28.3,0.167,112.3,...,22.7,0.222,94.2,1,1,10,3,2.0,14,2024
1,2,2023-10-26,1.0,108.0,101.7,0.500,15.6,35.4,0.137,106.2,...,17.0,0.274,99.2,0,0,10,3,2.0,24,2024
2,2,2023-10-27,1.0,108.0,94.4,0.558,12.8,18.9,0.286,114.4,...,22.0,0.213,111.7,1,1,10,4,2.0,2,2024
3,2,2023-10-27,1.0,124.0,98.3,0.665,14.3,14.7,0.183,126.2,...,16.3,0.185,99.6,0,1,10,4,2.0,21,2024
4,2,2023-10-27,1.0,103.0,93.7,0.446,6.3,29.1,0.228,110.0,...,32.0,0.137,119.8,0,0,10,4,2.0,16,2024


In [6]:
teams_df.to_excel('Models/train.xlsx', index=0)
current_df.to_excel('Models/test.xlsx', index=0)

In [16]:
date_df = teams_df.loc[(teams_df['Date'] == '2019-10-26')][['Date', 'TeamID', 'OppID', 'Pts', 'OppPts', 'Target']]
for team in date_df['TeamID'].unique():
    team_row = date_df.loc[date_df['TeamID'] == team]
    opp_row = date_df.loc[date_df['OppID'] == team]
    display(pd.concat([team_row, opp_row]))

,Date,TeamID,OppID,Pts,OppPts,Target
20,2019-10-26,11,19,111.0,119.0,1
35,2019-10-26,19,11,119.0,111.0,0


,Date,TeamID,OppID,Pts,OppPts,Target
21,2019-10-26,12,6,110.0,85.0,0
25,2019-10-26,6,12,85.0,110.0,1


,Date,TeamID,OppID,Pts,OppPts,Target
22,2019-10-26,20,2,110.0,102.5,0
36,2019-10-26,2,20,102.5,110.0,1


,Date,TeamID,OppID,Pts,OppPts,Target
23,2019-10-26,1,22,117.0,94.0,1
32,2019-10-26,22,1,94.0,117.0,0


,Date,TeamID,OppID,Pts,OppPts,Target
24,2019-10-26,26,29,103.5,93.0,0
28,2019-10-26,29,26,93.0,103.5,1


...

,Date,TeamID,OppID,Pts,OppPts,Target
35,2019-10-26,19,11,119.0,111.0,0
20,2019-10-26,11,19,111.0,119.0,1


,Date,TeamID,OppID,Pts,OppPts,Target
36,2019-10-26,2,20,102.5,110.0,1
22,2019-10-26,20,2,110.0,102.5,0


,Date,TeamID,OppID,Pts,OppPts,Target
37,2019-10-26,9,23,109.5,107.0,0
34,2019-10-26,23,9,107.0,109.5,1


,Date,TeamID,OppID,Pts,OppPts,Target
38,2019-10-26,28,4,118.0,117.5,1
39,2019-10-26,4,28,117.5,118.0,0


,Date,TeamID,OppID,Pts,OppPts,Target
39,2019-10-26,4,28,117.5,118.0,0
38,2019-10-26,28,4,118.0,117.5,1
